# Exemplo 13: Games - Leaderboard em Tempo Real (Spark)

Este notebook demonstra a criação de um ranking (leaderboard) de jogadores em tempo real usando **Spark Streaming**.

**Cenário**: Calcular a pontuação total acumulada de jogadores em janelas deslizantes.

## 1. Configuração

In [ ]:
# Instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar e Instalar Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz && tar xf spark-3.5.0-bin-hadoop3.tgz

# Baixar e Instalar Kafka
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz && tar xf kafka_2.13-3.6.1.tgz

# Instalar pacotes Python
!pip install -q findspark pyspark kafka-python

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
import findspark
findspark.init()

## 2. Iniciar Kafka

In [ ]:
%%bash
cd kafka_2.13-3.6.1
bin/zookeeper-server-start.sh -daemon config/zookeeper.properties
sleep 5
bin/kafka-server-start.sh -daemon config/server.properties
sleep 5
bin/kafka-topics.sh --create --topic gaming-scores --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1

## 3. Simulador de Pontuações

In [ ]:
import json
import time
import random
from kafka import KafkaProducer
import threading
from datetime import datetime

def generate_scores():
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))
    players = [f'Player_{i}' for i in range(1, 21)]
    
    try:
        for _ in range(500):
            data = {
                'player_id': random.choice(players),
                'score': random.randint(10, 1000),
                'timestamp': datetime.now().isoformat()
            }
            producer.send('gaming-scores', value=data)
            time.sleep(0.05)
    finally:
        producer.close()

thread = threading.Thread(target=generate_scores)
thread.start()

## 4. Leaderboard em Tempo Real

In [ ]:
%%writefile kafka_consumer.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, sum as _sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

spark = SparkSession.builder.appName("GamingLeaderboard").getOrCreate()

schema = StructType([
    StructField("player_id", StringType()),
    StructField("score", IntegerType()),
    StructField("timestamp", TimestampType())
])

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "gaming-scores") \
    .load()

scores = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

# Leaderboard cumulativo
leaderboard = scores.groupBy("player_id") \
    .agg(_sum("score").alias("total_score")) \
    .orderBy(col("total_score").desc())

# Note: Complete mode é necessário para agregações sem watermark
query = leaderboard.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()


query.awaitTermination()

In [ ]:
!spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 kafka_consumer.py